Sources Used:
- https://betfair-datascientists.github.io/api/apiPythontutorial/

Linux:
- http://www.bespokebots.com/betfair-ssl-certs.php

Windows:
- https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Non-Interactive+%28bot%29+login

In [6]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
import git

In [7]:
# loading in api login credentials
repo = git.Repo('.', search_parent_directories=True)
logins_dir = str(repo.working_tree_dir) + "/api_logins.json"

with open(logins_dir) as f:
    login_dict =  json.load(f)

In [8]:
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

In [9]:
trading.login()

<LoginResource>

In [10]:
# make event type request to find horse racing event type

horse_racing_event_type_id = trading.betting.list_event_types(
    filter=filters.market_filter(
        text_query='Horse Racing'
    )
)

for event_type in horse_racing_event_type_id:
    print(event_type.event_type.id, event_type.event_type.name, event_type.market_count) # prints id, name and market count
    horse_racing_id = event_type.event_type.id

    # list all horse racing market catalogues
    market_catalogues = trading.betting.list_market_catalogue(
        filter=filters.market_filter(
            event_type_ids=[horse_racing_id],  # filter on just horse racing
#             market_countries=['GB'],  # filter on just GB countries
            market_type_codes=['WIN'],  # filter on just WIN market types
        ),
        
        market_projection=['MARKET_START_TIME', 'RUNNER_DESCRIPTION'],  # runner description required
        max_results=100
    )

    print('%s market catalogues returned' % len(market_catalogues))

7 Horse Racing 221
100 market catalogues returned


In [11]:
def process_runner_books(runner_books):
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]

    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [12]:
market_IDs = []
market_start_times = []
market_names = []
runner_dict = {}

for market_catalogue in market_catalogues:
    market_IDs = market_IDs + [market_catalogue.market_id]
    market_names = market_names + [market_catalogue.market_name]
    market_start_times.append(market_catalogue.market_start_time)
    
    for runner in market_catalogue.runners:
        runner_dict[runner.selection_id] = runner.runner_name
    
GB_runners_df = pd.DataFrame()  # create an empty dataframe to append other dfs to

# market book request
for market,time, name in zip(market_IDs, market_start_times, market_names):
    
    # Create a price filter. Get all traded and offer data
    price_filter = filters.price_projection(
        price_data=['EX_BEST_OFFERS']
    )
    
    market_books = trading.betting.list_market_book(
        market_ids=[market],
        price_projection=price_filter
    )
    
    # append the new market book runners to the final dataframe
    market_book = market_books[0]
    runners_df = process_runner_books(market_book.runners)
    runners_df['Event_ID'] = market
    runners_df['Event Time'] = time
    runners_df['Event Name'] = name
    GB_runners_df = GB_runners_df.append(runners_df, ignore_index=True)

In [20]:
market_IDs;
market_start_times;
runner_dict;
market_names;

In [14]:
GB_runners_df['Selection Name'] = GB_runners_df['Selection ID'].map(runner_dict)

In [260]:
# GB_runners_df.head(30)

In [ ]:
# include irish races as well?

In [ ]:
# creating df for each event and appending to list - > producing list of dataframes for each event

In [15]:
# speed this up - there are ways quicker than for loop but see what is needed first. 
event_dfs = []

for event_id in market_IDs:
    event_dfs.append(GB_runners_df.loc[GB_runners_df['Event_ID'] == event_id])
    

In [18]:
GB_runners_df

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor,Event_ID,Event Time,Event Name,Selection Name
0,19156710,9.20,54.15,11.00,13.36,9.2,0.0,ACTIVE,None,12.075,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,1. Quite a Quality
1,28355545,1.92,78.05,1.93,36.62,1.92,0.0,ACTIVE,None,36.225,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,2. Love Haleryd
2,28355546,10.50,15.87,12.50,14.16,11.5,0.0,ACTIVE,None,6.586,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,3. Vulcano O.P.
3,28355547,10.00,28.11,20.00,10.56,11.5,0.0,ACTIVE,None,12.500,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,4. Iwan Boko
4,18105690,46.00,17.38,550.00,13.01,90,0.0,ACTIVE,None,1.818,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,5. Floyd Fortuna
5,13430166,36.00,11.01,100.00,12.40,50,0.0,ACTIVE,None,4.545,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,6. Vagabond Vik
6,28355548,25.00,10.23,50.00,10.28,27,0.0,ACTIVE,None,6.250,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,7. Powderfinger
7,28355549,6.80,13.74,7.60,15.73,7.2,0.0,ACTIVE,None,20.000,1.170314130,2020-04-20 10:20:00,R1 2640m Trot S,8. Digital Science
8,25375326,12.50,64.59,65.00,12.16,13.5,0.0,ACTIVE,None,2.083,1.170311382,2020-04-20 10:32:00,R6 1790m Pace M,1. Urella Fella
9,5658711,19.00,22.41,130.00,15.41,20,0.0,ACTIVE,None,10.867,1.170311382,2020-04-20 10:32:00,R6 1790m Pace M,2. Thelonghaul
